# Processing with Multiple Linear Regression Algorithm

### Import Library

In [3]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.linear_model import Ridge
from sklearn.model_selection import GridSearchCV
import joblib
import os

### Membaca Data hasil Pre-Processing

In [5]:
df = pd.read_csv('D:/FOLDER MUTIARA/SEMESTER 5/PM/Project/Analisis-faktor-programming-skill/data/hasil_preprocessing2.csv', sep=',')

### Melihat 5 Baris Pertama Data

Menggunakan df.head() untuk melihat 5 baris pertama dari dataset. Ini memberikan gambaran umum tentang struktur data.

In [8]:
df.head()

,Durasi Laptop,Penggunaan Medsos di Laptop,Durasi Medsos di Laptop,Social_Media_Instagram di Laptop,Social_Media_Tiktok di Laptop,Social_Media_Youtube di Laptop,Hiburan Instagram di laptop,Berita Instagram di laptop,Edukasi Instagram di laptop,Teknologi Instagram di laptop,Hobi Instagram di laptop,Hiburan TikTok di laptop,Berita TikTok di laptop,Edukasi TikTok di laptop,Teknologi TikTok di laptop,Hobi TikTok di laptop,Programming Skill
0,0.591079,1.241025,0.555879,1.406542,0.891131,-1.025417,0.512125,0.356383,0.477765,0.963299,-0.012744,0.644919,0.559650,0.676490,0.808165,0.269818,70
1,-0.456171,0.374851,-1.971706,-0.179784,0.196086,-0.018519,0.436638,0.452571,-0.957009,-0.355315,-0.314228,0.183296,0.191300,-0.977658,-0.522651,0.798145,42
2,-0.799710,-0.022879,-0.205504,1.021602,1.068378,-0.746934,-0.031310,0.296527,0.161031,-0.811833,-0.547288,0.627937,0.805289,0.024191,-0.577333,-0.060597,36
3,0.050544,-1.508204,-0.846895,0.676422,-0.111780,-0.883244,-0.255622,-1.562992,0.198643,0.375544,0.657302,-0.171491,-0.238071,-0.132331,0.043824,0.089143,56
4,-2.473920,0.418309,0.189722,-0.859500,0.322300,-0.493285,0.707085,-0.069923,0.229330,0.375864,0.744495,0.243720,-0.373004,0.204117,-0.189017,-0.136902,76


### Membagi Data Menjadi Training, Validation, dan Testing

kita akan membagi dataset menjadi tiga bagian: training set, validation set, dan test set. Tujuan dari pembagian ini adalah agar kita bisa melatih model, mengevaluasi model selama training, dan menguji kemampuan model 

In [11]:
X = df.drop('Programming Skill', axis=1)  # Semua kolom fitur
y = df['Programming Skill']  # Target variabel

# Membagi data menjadi 60% training, 20% validation, dan 20% testing
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.4, random_state=42)

# Membagi sisa (X_temp dan y_temp) menjadi validation dan testing (50% masing-masing)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

# Mengecek ukuran data
print(f'Training set: {X_train.shape}, Validation set: {X_val.shape}, Test set: {X_test.shape}')

Training set: (303, 16), Validation set: (101, 16), Test set: (102, 16)


### Membangun Model Multiple Linear Regression

Setelah data dibagi, kita akan membangun model regresi linear menggunakan scikit-learn.

In [14]:
# Membuat model Multiple Linear Regression
model = LinearRegression()

In [15]:
# K-fold Cross-validation
scores = cross_val_score(model, X_train, y_train, cv=5, scoring='neg_mean_squared_error')

# Menghitung MSE rata-rata dari cross-validation
mean_mse = np.mean(-scores)
print(f'Mean Cross-Validated MSE: {mean_mse}')


Mean Cross-Validated MSE: 187.2154369357993


In [16]:
# Melatih model dengan training data
model.fit(X_train, y_train)

LinearRegression()

### Hyperparameter Tuning

hyperparameter tuning menggunakan GridSearchCV atau RandomizedSearchCV untuk mencari hyperparameter terbaik dari model.

In [19]:
# Tentukan model Ridge
ridge_model = Ridge()

# Tentukan grid hyperparameter
param_grid = {'alpha': [0.1, 1, 10, 100, 1000]}

# Melakukan Grid Search dengan cross-validation
grid_search = GridSearchCV(estimator=ridge_model, param_grid=param_grid, cv=5, scoring='neg_mean_squared_error')
grid_search.fit(X_train, y_train)

# Menampilkan hasil grid search
print(f'Best hyperparameter alpha: {grid_search.best_params_}')
print(f'Best cross-validated score (MSE): {grid_search.best_score_}')

# Menggunakan model dengan hyperparameter terbaik
best_model = grid_search.best_estimator_


Best hyperparameter alpha: {'alpha': 1000}
Best cross-validated score (MSE): -169.56141792339344


### Evaluasi Model pada Validation dan Test Set

In [21]:
from sklearn.metrics import mean_squared_error, r2_score

# Evaluasi dengan Validation Set
y_val_pred = best_model.predict(X_val)
validation_mse = mean_squared_error(y_val, y_val_pred)
validation_r2 = r2_score(y_val, y_val_pred)

print(f'Validation MSE: {validation_mse}')
print(f'Validation R²: {validation_r2}')

# Evaluasi dengan Test Set
y_test_pred = best_model.predict(X_test)
test_mse = mean_squared_error(y_test, y_test_pred)
test_r2 = r2_score(y_test, y_test_pred)

print(f'Test MSE: {test_mse}')
print(f'Test R²: {test_r2}')


Validation MSE: 160.99376187184015
Validation R²: -0.015815501961147005
Test MSE: 187.80874645612994
Test R²: -0.018889554787531404


In [22]:
input_data = {
    "Durasi Laptop": 0.591078974194839,
    "Penggunaan Medsos di Laptop": 1.2410247596063955,
    "Durasi Medsos di Laptop": 0.5558793479263375,
    "Social_Media_Instagram di Laptop": 1.4065416168256142,
    "Social_Media_Tiktok di Laptop": 0.8911313706714228,
    "Social_Media_Youtube di Laptop": -1.0254173370887756,
    "Hiburan Instagram di laptop": 0.5121248660448603,
    "Berita Instagram di laptop": 0.35638270848626813,
    "Edukasi Instagram di laptop": 0.47776512593518883,
    "Teknologi Instagram di laptop": 0.963299130025909,
    "Hobi Instagram di laptop": -0.012743816714464748,
    "Hiburan TikTok di laptop": 0.6449185930417787,
    "Berita TikTok di laptop": 0.5596503971054282,
    "Edukasi TikTok di laptop": 0.6764898031864922,
    "Teknologi TikTok di laptop": 0.8081649617155,
    "Hobi TikTok di laptop": 0.2698184435933873
}

# Mengubah data input ke dalam format DataFrame
df_input = pd.DataFrame([input_data])

In [23]:
# Melakukan prediksi menggunakan model
prediksi = model.predict(df_input)

# Menampilkan hasil prediksi
print(f'Hasil Prediksi: {prediksi[0]}')

Hasil Prediksi: 62.83327489988184


### Evaluasi Model dengan Validation Set

Setelah melatih model, kita akan menguji model pada data validation untuk mengevaluasi kinerja model selama training.

In [26]:
# # Memprediksi nilai target untuk data validation
# y_val_pred = model.predict(X_val)

# # Menghitung MSE dan R-squared pada data validation
# mse_val = mean_squared_error(y_val, y_val_pred)
# r2_val = r2_score(y_val, y_val_pred)

# # Menampilkan hasil evaluasi
# print(f'Validation MSE: {mse_val}')
# print(f'Validation R-squared: {r2_val}')

### Evaluasi Model dengan Test Set

In [28]:
# # Memprediksi nilai target untuk data testing
# y_test_pred = model.predict(X_test)

# # Menghitung MSE dan R-squared pada data testing
# mse_test = mean_squared_error(y_test, y_test_pred)
# r2_test = r2_score(y_test, y_test_pred)

# # Menampilkan hasil evaluasi pada testing set
# print(f'Test MSE: {mse_test}')
# print(f'Test R-squared: {r2_test}')

In [29]:
# Tentukan path lengkap untuk menyimpan model
model_path = 'D:/FOLDER MUTIARA/SEMESTER 5/PM/Project/Analisis-faktor-programming-skill/results/multiple_linear_regression_model.pkl'

# Menyimpan model yang sudah dilatih ke dalam file
joblib.dump(model, model_path)

print(f'Model telah disimpan di: {model_path}')

Model telah disimpan di: D:/FOLDER MUTIARA/SEMESTER 5/PM/Project/Analisis-faktor-programming-skill/results/multiple_linear_regression_model.pkl


In [30]:
# # Memuat model yang sudah disimpan
# import joblib

# model = joblib.load('D:/FOLDER MUTIARA/SEMESTER 5/PM/Project/Analisis-faktor-programming-skill/results/multiple_linear_regression_model.pkl')


**Dalam multiple linear regression, kita tidak menggunakan akurasi dan confusion matrix karena kedua metrik tersebut lebih relevan untuk model klasifikasi. Berikut penjelasannya:**

**1. Akurasi**
Akurasi adalah metrik yang digunakan untuk menilai kinerja model klasifikasi. Ini mengukur proporsi prediksi yang benar dari total prediksi.
Dalam regresi, kita tidak memiliki kategori untuk diprediksi. Sebaliknya, kita memprediksi nilai kontinu, sehingga akurasi tidak dapat diterapkan.<br>
**2. Confusion Matrix**
Confusion matrix adalah alat untuk mengevaluasi kinerja model klasifikasi dengan menunjukkan jumlah prediksi yang benar dan salah untuk setiap kelas.Untuk regresi, kita tidak memiliki kelas. Alih-alih, kita memprediksi nilai, jadi confusion matrix tidak relevan.
Metrik yang Digunakan dalam Regresi <br>

Dalam regresi, beberapa metrik yang umum digunakan untuk menilai kinerja model meliputi:<br>

**Mean Absolute Error (MAE)**: Rata-rata dari selisih absolut antara nilai yang diprediksi dan nilai aktual.<br>
**Mean Squared Error (MSE)**: Rata-rata dari kuadrat selisih antara nilai yang diprediksi dan nilai aktual.<br>
**Root Mean Squared Error (RMSE)**: Akar kuadrat dari MSE, memberikan ukuran kesalahan dalam satuan yang sama dengan nilai yang diprediksi.<br>
**R-squared (R²)**: Mengukur proporsi variabilitas dalam data yang dapat dijelaskan oleh model.<br>